In [1]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

# TODO: Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment1/'
FOLDERNAME = "LG Aimers Phase2"
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append(f'/content/drive/My Drive/{FOLDERNAME}')

%cd /content/drive/My\ Drive/$FOLDERNAME

ModuleNotFoundError: No module named 'google'

In [9]:
%matplotlib inline
import time
import pandas as pd
import torch
from torch import nn
import torch.optim as optim
import numpy as np
import copy
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

train_df = pd.read_csv(f'/content/drive/My Drive/{FOLDERNAME}/data/train.csv').sample(frac=1, random_state=0)
test_df = pd.read_csv(f'/content/drive/My Drive/{FOLDERNAME}/data/test.csv')

In [10]:
def preprocess(train_df, test_df):
    # Remove the ID and label columns
    X = train_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality'])
    y = train_df['Y_Class']

    test = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

    qual_col = ['LINE', 'PRODUCT_CODE']

    for i in qual_col:
        le = LabelEncoder()
        le = le.fit(X[i])
        X[i] = le.transform(X[i])
        
        for label in np.unique(test[i]): 
            if label not in le.classes_: 
                le.classes_ = np.append(le.classes_, label)
        test[i] = le.transform(test[i])

    X = X.fillna(0)
    test = test.fillna(0)

    return X, y, test

In [11]:
X, y, test = preprocess(train_df, test_df)

In [29]:
from sklearn.model_selection import StratifiedKFold
random_state = 2
N_SPLITS = 4

skf = StratifiedKFold(n_splits=N_SPLITS) # 20%씩 validation data가 됩니다.

# XGB

In [13]:
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
import numpy as np
folds = skf.split(X, y)
x_scores = np.empty(N_SPLITS)
for idx, (train_idx, test_idx) in enumerate(folds):
    print("=" * 12 + f"Training fold {idx}" + 12 * "=")
    start = time.time()

    X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_val = y[train_idx], y[test_idx]
    eval_set = [(X_val, y_val)]

    xgb_clf = XGBClassifier(n_estimators=10000, max_depth=9, tree_method="hist",
                            learning_rate=0.05, objective='multi:softprob',
                            scale_pos_weight=7, random_state=random_state)
    xgb_clf.fit(
        X_train,
        y_train,
        eval_set=eval_set,
        early_stopping_rounds=200,
    )

    preds = xgb_clf.predict(X_val)
    loss = f1_score(y_val, preds, average='macro')
    x_scores[idx] = loss
    runtime = time.time() - start
    print(f"Fold {idx} finished with score: {loss:.5f} in {runtime:.2f} seconds.\n")

============Training fold 0============
[0]	validation_0-merror:0.493333
Will train until validation_0-merror hasn't improved in 200 rounds.
[1]	validation_0-merror:0.48
[2]	validation_0-merror:0.473333
[3]	validation_0-merror:0.46
[4]	validation_0-merror:0.453333
[5]	validation_0-merror:0.446667
[6]	validation_0-merror:0.453333
[7]	validation_0-merror:0.46
[8]	validation_0-merror:0.46
[9]	validation_0-merror:0.46
[10]	validation_0-merror:0.44
[11]	validation_0-merror:0.44
[12]	validation_0-merror:0.433333
[13]	validation_0-merror:0.44
[14]	validation_0-merror:0.433333
[15]	validation_0-merror:0.453333
[16]	validation_0-merror:0.446667
[17]	validation_0-merror:0.446667
[18]	validation_0-merror:0.446667
[19]	validation_0-merror:0.446667
[20]	validation_0-merror:0.433333
[21]	validation_0-merror:0.44
[22]	validation_0-merror:0.426667
[23]	validation_0-merror:0.426667
[24]	validation_0-merror:0.44
[25]	validation_0-merror:0.44
[26]	validation_0-merror:0.433333
[27]	validation_0-merror:0.4

# LGBM

In [21]:
!pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
import lightgbm as lgbm
folds = skf.split(X, y)
l_scores = np.empty(N_SPLITS)
for idx, (train_idx, test_idx) in enumerate(folds):
    print("=" * 12 + f"Training fold {idx}" + 12 * "=")
    start = time.time()

    X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_val = y[train_idx], y[test_idx]
    eval_set = [(X_val, y_val)]

    lgbm_clf = lgbm.LGBMClassifier(n_estimators=10000, max_depth=9, num_leaves=80, min_data_in_leaf = 10,
                                   bagging_freq=5, bagging_fraction=0.75, feature_fraction=0.5,
                                   min_gain_to_split=5,
                                   random_state=random_state)
    lgbm_clf.fit(
        X_train,
        y_train,
        eval_set=eval_set,
        early_stopping_rounds=200
    )

    preds = lgbm_clf.predict_proba(X_val)
    loss = f1_score(y_val, np.argmax(preds, axis=1), average='macro')
    l_scores[idx] = loss
    runtime = time.time() - start
    print(f"Fold {idx} finished with score: {loss:.5f} in {runtime:.2f} seconds.\n")

============Training fold 0============
[1]	valid_0's multi_logloss: 1.02053
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's multi_logloss: 1.02224
[3]	valid_0's multi_logloss: 1.02224
[4]	valid_0's multi_logloss: 1.02224
[5]	valid_0's multi_logloss: 1.02328
[6]	valid_0's multi_logloss: 1.02328
[7]	valid_0's multi_logloss: 1.02328
[8]	valid_0's multi_logloss: 1.02328
[9]	valid_0's multi_logloss: 1.02328
[10]	valid_0's multi_logloss: 1.02502
[11]	valid_0's multi_logloss: 1.02502
[12]	valid_0's multi_logloss: 1.02502
[13]	valid_0's multi_logloss: 1.02502
[14]	valid_0's multi_logloss: 1.02502
[15]	valid_0's multi_logloss: 1.02502
[16]	valid_0's multi_logloss: 1.02502
[17]	valid_0's multi_logloss: 1.02502
[18]	valid_0's multi_logloss: 1.02502
[19]	valid_0's multi_logloss: 1.02502
[20]	valid_0's multi_logloss: 1.02502
[21]	valid_0's multi_logloss: 1.02502
[22]	valid_0's multi_logloss: 1.02502
[23]	valid_0's multi_logloss: 1.02502
[24]	valid_0's multi_logloss: 1.

# AdaBoost

In [25]:
from sklearn.ensemble import AdaBoostClassifier
folds = skf.split(X, y)
a_scores = np.empty(N_SPLITS)
for idx, (train_idx, test_idx) in enumerate(folds):
    print("=" * 12 + f"Training fold {idx}" + 12 * "=")
    start = time.time()

    X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_val = y[train_idx], y[test_idx]
    eval_set = [(X_val, y_val)]

    ad_clf = AdaBoostClassifier(n_estimators=1000, learning_rate=0.03, random_state=random_state)
    ad_clf.fit(
        X_train,
        y_train
    )

    preds = ad_clf.predict_proba(X_val)
    loss = f1_score(y_val, preds)
    a_scores[idx] = loss
    runtime = time.time() - start
    print(f"Fold {idx} finished with score: {loss:.5f} in {runtime:.2f} seconds.\n")

============Training fold 0============
Fold 0 finished with score: 1.08848 in 44.84 seconds.

============Training fold 1============
Fold 1 finished with score: 1.05809 in 43.63 seconds.

============Training fold 2============
Fold 2 finished with score: 1.05599 in 41.71 seconds.

============Training fold 3============
Fold 3 finished with score: 1.07083 in 41.79 seconds.

============Training fold 4============
Fold 4 finished with score: 1.05413 in 41.72 seconds.

============Training fold 5============
Fold 5 finished with score: 1.05305 in 40.69 seconds.

============Training fold 6============
Fold 6 finished with score: 1.08496 in 42.35 seconds.



# TabNet

In [1]:
!pip install pytorch-tabnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 KB 2.0 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.tab_model import TabNetModel

class TabNetBClassifier(nn.Module):
    def __init__(self, num_features, num_classes):
        super(TabNetBClassifier, self).__init__()
        
        self.tabnet = TabNetModel(
            input_dim= ,
            output_dim= ,
            n_d= ,
            n_a= ,
            n_steps= ,
            gamma= ,
            lambda_sparse= ,
            momentum= ,
            clip_value= ,
            optimizer_fn=torch.optim. ,
            optimizer_params=dict(lr= ),
            mask_type=" ",
            scheduler_params=dict(
                mode=" ",
                patience= ,
                min_lr= ,
                factor= ,
                verbose=True
            )
        )

    def forward(self, x):
      
        return self.tabnet(x)

In [ ]:
model = TabNetBClassifier()

criterion = torch.nn.()

epochs = 

for epoch in range(epochs):
    for x, y in train_loader:
        optimizer.zero_grad()
        output = model(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        valid_loss = 0
        for x, y in valid_loader:
            output = model(x)
            loss = criterion(output, y)
            valid_loss += loss.item()
        valid_loss /= len(valid_loader)

    print(f"Epoch {epoch+1} loss: {valid_loss}")